In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import numpy as np
import pandas as pd
import statsmodels.api as sm
from datetime import date
from scipy import stats
from sklearn.metrics import mean_squared_error
from math import sqrt
from random import randint
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import GRU
from keras.callbacks import EarlyStopping
from keras import initializers
from matplotlib import pyplot
from datetime import datetime
from matplotlib import pyplot as plt
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
from sklearn.svm import SVR 
import matplotlib.pyplot as plt 
import pandas as pd 
import plotly.graph_objs as go
%matplotlib inline

In [ ]:

data = pd.read_csv('../input/bitcoin-historical-data/bitstampUSD_1-min_data_2012-01-01_to_2020-09-14.csv')
data.isnull().values.any() #Will print False in case of no null values
data.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)
data['date'] = pd.to_datetime(data['Timestamp'],unit='s').dt.date
group = data.groupby('date')
Daily_Price = group['Weighted_Price'].mean()

Daily_Price.head()

In [ ]:
from datetime import date
d0 = date(2017, 1, 1)
d1 = date(2020, 9, 9)
delta = d1 - d0
days_look = delta.days + 1
print(days_look)

d0 = date(2019, 8, 21)
d1 = date(2020, 9 , 14)
delta = d1 - d0
days_from_train = delta.days + 1
print(days_from_train)

d0 = date(2020, 9, 9)
d1 = date(2020, 9, 14)
delta = d1 - d0
days_from_end = delta.days + 1
print(days_from_end)

In [ ]:
df_train= Daily_Price[len(Daily_Price)-days_look-days_from_end:len(Daily_Price)-days_from_train]
df_test= Daily_Price[len(Daily_Price)-days_from_train:]
print(len(df_train), len(df_test))

In [ ]:
def create_lookback(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset) - look_back):
        a = dataset[i:(i + look_back), 0]
        X.append(a)
        Y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(Y)

In [ ]:
training_set = df_train.values
training_set = np.reshape(training_set, (len(training_set), 1))
test_set = df_test.values
test_set = np.reshape(test_set, (len(test_set), 1))
from sklearn.preprocessing import MinMaxScaler
#scale datasets
scaler = MinMaxScaler()
training_set = scaler.fit_transform(training_set)
test_set = scaler.transform(test_set)

# create datasets which are suitable for time series forecasting
look_back = 1
X_train, Y_train = create_lookback(training_set, look_back)
X_test, Y_test = create_lookback(test_set, look_back)

 # reshape datasets so that they will be ok for the requirements of the LSTM model in Keras
#X_train = np.reshape(X_train, (len(X_train), 1, X_train.shape[1]))
#X_test = np.reshape(X_test, (len(X_test), 1, X_test.shape[1]))
print(Y_test)

In [ ]:
svr_lin  = SVR(kernel='linear')
svr_poly = SVR(kernel='poly')
svr_rbf = SVR(kernel='rbf')
    

In [ ]:
svr_lin.fit(X_train, Y_train)
svr_poly.fit(X_train, Y_train)
svr_rbf.fit(X_train, Y_train)

In [ ]:
# add one additional data point to align shapes of the predictions and true labels
#print(X_test.shape)
#print(working_data.shape)
#y=scaler.transform(working_data.iloc[-1][0].reshape(1,-1))

#X_test = np.append(X_test, y)
#X_test = np.reshape(X_test, (len(X_test), 1))
#print(X_test.shape)
print("ok")
# get predictions and then make some transformations to be able to calculate RMSE properly in USD
prediction_lin = svr_lin.predict(X_test)
prediction_inverse_lin = scaler.inverse_transform(prediction_lin.reshape(-1, 1))
Y_test_inverse_lin = scaler.inverse_transform(Y_test.reshape(-1, 1))
prediction2_inverse_lin = np.array(prediction_inverse_lin[:,0][1:])
Y_test2_inverse_lin = np.array(Y_test_inverse_lin[:,0])

prediction_poly = svr_poly.predict(X_test)
prediction_inverse_poly = scaler.inverse_transform(prediction_poly.reshape(-1, 1))
Y_test_inverse_poly = scaler.inverse_transform(Y_test.reshape(-1, 1))
prediction2_inverse_poly = np.array(prediction_inverse_poly[:,0][1:])
Y_test2_inverse_poly = np.array(Y_test_inverse_poly[:,0])

prediction_rbf = svr_rbf.predict(X_test)
prediction_inverse_rbf = scaler.inverse_transform(prediction_rbf.reshape(-1, 1))
Y_test_inverse_rbf = scaler.inverse_transform(Y_test.reshape(-1, 1))
prediction2_inverse_rbf = np.array(prediction_inverse_rbf[:,0][1:])
Y_test2_inverse_rbf = np.array(Y_test_inverse_rbf[:,0])

In [ ]:
trace1 = go.Scatter(
    x = np.arange(0, len(prediction2_inverse_lin), 1),
    y = prediction2_inverse_lin,
    mode = 'lines',
    name = 'Linear Predicted labels',
    line = dict(color=('rgb(255, 0, 0)'), width=2)
)
trace2 = go.Scatter(
    x = np.arange(0, len(prediction2_inverse_poly), 1),
    y = prediction2_inverse_poly,
    mode = 'lines',
    name = 'Polynomial Predicted labels',
    line = dict(color=('rgb(0, 255, 0)'), width=2)
)
trace3 = go.Scatter(
    x = np.arange(0, len(prediction2_inverse_rbf), 1),
    y = prediction2_inverse_rbf,
    mode = 'lines',
    name = 'RBF Predicted labels',
    line = dict(color=('rgb(0, 0, 255)'), width=2)
)
trace4 = go.Scatter(
    x = np.arange(0, len(Y_test2_inverse_lin), 1),
    y = Y_test2_inverse_lin,
    mode = 'lines',
    name = 'True labels',
    line = dict(color=('rgb(0,0,0)'), width=2)
)

data = [trace1, trace2, trace3, trace4]
layout = dict(title = 'Comparison of true prices (on the test dataset) with prices our model predicted',
             xaxis = dict(title = 'Day number'), yaxis = dict(title = 'Price, USD'))
fig = dict(data=data, layout=layout)
py.iplot(fig, filename='results_demonstrating0')

In [ ]:
Test_Dates = Daily_Price[len(Daily_Price)-days_from_train:].index

trace1 = go.Scatter(x=Test_Dates, y=Y_test2_inverse_lin, name= 'Actual Price',
                   line = dict(color = ('rgb(0, 0, 0)'),width = 2))
trace2 = go.Scatter(x=Test_Dates, y=prediction2_inverse_lin, name= 'Predicted Price Linear',
                   line = dict(color = ('rgb(255, 0, 0)'),width = 2))
trace3 = go.Scatter(x=Test_Dates, y=prediction2_inverse_poly, name= 'Predicted Price polynomial',
                   line = dict(color = ('rgb(0, 255, 0)'),width = 2))
trace4 = go.Scatter(x=Test_Dates, y=prediction2_inverse_rbf, name= 'Predicted Price RBF',
                   line = dict(color = ('rgb(0, 0, 255)'),width = 2))
data = [trace1, trace2, trace3, trace4]
layout = dict(title = 'Comparison of true prices (on the test dataset) with prices our model predicted, by dates',
             xaxis = dict(title = 'Date'), yaxis = dict(title = 'Price, USD'))
fig = dict(data=data, layout=layout)
py.iplot(fig, filename='results_demonstrating1')

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
score = svr_lin.score(X_test,Y_test)
print("Linear SVR")
print("R-squared:", score)
print("RMSE:", sqrt(mean_squared_error(Y_test, svr_lin.predict(X_test))))
print("Mean absolute error:", mean_absolute_error(Y_test, svr_lin.predict(X_test)))
score = svr_poly.score(X_test,Y_test)
print("Poly SVR")
print("R-squared:", score)
print("RMSE:",sqrt (mean_squared_error(Y_test, svr_poly.predict(X_test))))
print("Mean absolute error:", mean_absolute_error(Y_test,svr_poly.predict(X_test)))
score = svr_rbf.score(X_test,Y_test)
print("Rbf SVR")
print("R-squared:", score)
print("RMSE:", sqrt(mean_squared_error(Y_test, svr_rbf.predict(X_test))))
print("Mean absolute error:", mean_absolute_error(Y_test, svr_rbf.predict(X_test)))


In [ ]:
from sklearn.linear_model import BayesianRidge

gnb = BayesianRidge()
gnb.fit(X_train, Y_train)
print(gnb.score(X_test,Y_test))
print("RMSE:", sqrt(mean_squared_error(Y_test, gnb.predict(X_test))))
print("Mean absolute error:", mean_absolute_error(Y_test, gnb.predict(X_test)))


In [ ]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train, Y_train)
print(lr.score(X_test, Y_test))
print("RMSE:", sqrt(mean_squared_error(Y_test, lr.predict(X_test))))
print("Mean absolute error:", mean_absolute_error(Y_test, lr.predict(X_test)))